# 1. 理解 Python 函数

In [ ]:
def foo():
    return "I am Le0v1n"

print(f"foo(): {foo()}")


fn = foo  # 这里 foo 后面没有小括号，不是函数调用，而是将 foo 函数赋值给变量 fn
print(f"fn(): {fn()}")

In [ ]:
def foo():
    print("foo 函数正在运行...")
    
    # 定义函数中的函数
    def bar():
        return "foo.bar 函数正在运行..."
    
    def bam():
        return "foo.bam 函数正在运行..."
        
    # 调用函数中的函数
    print(bar())
    print(bam())
    print("foo 函数即将结束!")
    

if __name__ == "__main__":
    foo()
    
    # 如果我们调用函数中的函数
    try:
        bar()
    except Exception as e:
        print(f"报错啦: {e}")
        
    try:
        bam()
    except Exception as e:
        print(f"报错啦: {e}")

In [ ]:
def foo(choice='bar'):
    print("foo 函数正在运行...")

    # 定义函数中的函数
    def bar():
        return "foo.bar 函数正在运行..."

    def bam():
        return "foo.bam 函数正在运行..."

    print("foo 函数即将结束!")

    if choice == 'bar':
        return bar
    elif choice == 'bam':
        return bam
    else:
        raise NotImplementedError("choice 必须是 bar 或 bam !")


if __name__ == "__main__":
    fn1 = foo(choice='bar')
    fn2 = foo(choice='bam')
    print(fn1)
    print(fn2)
    print(fn1())
    print(fn2())


In [ ]:
def foo():
    return "I am foo"

def bar(fn):
    print("I am bar")
    print(fn())
    
    
if __name__ == "__main__":
    bar(foo)
    print()
    
    try:
        bar(foo())
    except Exception as e:
        print(f"报错啦: {e}")

In [ ]:
def decorator(fn):
    def wrapper():
        print("---------- 函数调用前 ----------")
        fn()  # 调用函数
        print("---------- 函数调用后 ----------")
    return wrapper


def foo():
    print("I am foo!")
    
    
if __name__ == "__main__":
    # 直接调用函数
    print("直接调用函数: ", end="")
    foo()
    print()

    # 调用装饰器包装后函数
    fn = decorator(foo)  # 将foo函数用装饰器包装 -> fn
    print("调用包装后的foo函数: ")
    fn()  # 调用包装后的foo函数
    print()

# 2. 理解Python装饰器

In [ ]:
def decorator(fn):
    def wrapper():
        print("---------- 函数调用前 ----------")
        fn()
        print("---------- 函数调用后 ----------")

    return wrapper
    

@decorator  # @装饰器名称
def foo():
    print("I am foo")
    
    
if __name__ == "__main__":
    # 直接调用函数
    print("直接调用函数: ")
    foo()
    print()

    print(f"函数的名称: {foo.__name__}")

In [ ]:
from functools import wraps


def decorator(fn):
    @wraps(fn)
    def wrapper():
        print("---------- 函数调用前 ----------")
        fn()
        print("---------- 函数调用后 ----------")
    return wrapper


@decorator
def foo():
    print("I am foo")


if __name__ == "__main__":
    print("直接调用函数: ")
    foo()
    print()

    print(f"函数的名称: {foo.__name__}")

In [ ]:
from functools import wraps


class Decorate:
    def __init__(self, fn) -> None:
        self.fn = fn

    def __call__(self):
        @wraps(self.fn)
        def wrapper(*args, **kwargs):
            print("---------- 函数调用前 ----------")
            self.fn(*args, **kwargs)
            print("---------- 函数调用后 ----------")
        return wrapper


@Decorate  # 用类来装饰函数，那么函数也变为了类
def foo(param1, param2):
    print(f"I am foo. \n"
          f"My parameters are: \n"
          f"param1: {param1} | param2: {param2}")


if __name__ == "__main__":
    # 实例化类对象
    obj = foo()

    # 调用对象的方法
    obj("参数1", "参数2")


# 3. Python 注册器 Registry

In [ ]:
def foo():
    ...


def fn(x): return x**2


class ExampleClass:
    ...


if __name__ == "__main__":
    # 创建注册字典
    register_obj = dict()

    # 开始为函数和类进行注册
    register_obj[foo.__name__] = foo
    register_obj[fn.__name__] = fn
    register_obj[ExampleClass.__name__] = ExampleClass

    print(register_obj)


In [ ]:
class Register(dict):
    def __init__(self, *args, **kwargs):
        super(Register, self).__init__(*args, **kwargs)
        self._dict = dict()  # 创建一个字典用于保存注册的可调用对象

    def register(self, target):
        def add_item(key, value):
            if key in self._dict:  # 如果 key 已经存在
                print(f"\033[34m"
                      f"WARNING: {value.__name__} 已经存在!"
                      f"\033[0m")

            # 进行注册，将 key 和 value 添加到字典中
            self[key] = value
            return value

        # 传入的 target 可调用 --> 没有给注册名 --> 传入的函数名或类名作为注册名
        if callable(target):  # key 为函数/类的名称; value 为函数/类本体
            return add_item(key=target.__name__, value=target)
        else:  # 传入的 target 不可调用 --> 抛出异常
            raise TypeError("\033[31mOnly support callable object, e.g. function or class\033[0m")

    def __setitem__(self, key, value):  # 将键值对添加到 _dict 字典中
        self._dict[key] = value

    def __getitem__(self, key):  # 从 _dict 字典中获取注册的可调用对象
        return self._dict[key]

    def __contains__(self, key):  # 检查给定的注册名是否存在于 _dict 字典中
        return key in self._dict

    def __str__(self):  # 返回 _dict 字典的字符串表示
        return str(self._dict)

    def keys(self):  # 返回 _dict 字典中的所有键
        return self._dict.keys()

    def values(self):  # 返回 _dict 字典中的所有值
        return self._dict.values()

    def items(self):  # 返回 _dict 字典中的所有键值对
        return self._dict.items()


if __name__ == "__main__":
    register_obj = Register()
    
    @register_obj.register
    def fn1_add(a, b):
        return a + b
    
    @register_obj.register
    def fn2_subject(a, b):
        return a - b
    
    @register_obj.register
    def fn3_multiply(a, b):
        return a * b
    
    @register_obj.register
    def fn4_divide(a, b):
        return a / b
    
    # 我们再重复定义一个函数
    @register_obj.register
    def fn2_subject(a, b):
        return b - a
    
    # 尝试使用 register 方法注册不可调用的对象
    try:
        register_obj.register("传入字符串，它是不可调用的")
    except Exception as e:
        print(f"报错啦: {e}")

    print("\n所有函数均已注册!\n")
    
    # 我们查看一个注册器中有哪些元素
    print(f"\033[34mkey\t\tvalue\033[0m")
    for k, v in register_obj.items():  # <=> for k, v in register_obj._dict.items()
        print(f"{k}: \t{v}")

In [ ]:
class Register(dict):
    def __init__(self, *args, **kwargs):
        super(Register, self).__init__(*args, **kwargs)
        self._dict = dict()  # 创建一个字典用于保存注册的可调用对象

    def register(self, target):
        def add_item(key, value):
            if key in self._dict:  # 如果 key 已经存在
                print(f"\033[34m"
                      f"WARNING: {value.__name__} 已经存在!"
                      f"\033[0m")

            # 进行注册，将 key 和 value 添加到字典中
            self[key] = value
            return value

        # 传入的 target 可调用 --> 没有给注册名 --> 传入的函数名或类名作为注册名
        if callable(target):  # key 为函数/类的名称; value 为函数/类本体
            return add_item(key=target.__name__, value=target)
        else:  # 传入的 target 不可调用 --> 抛出异常
            raise TypeError("\033[31mOnly support callable object, e.g. function or class\033[0m")
        
    def __call__(self, target):
        return self.register(target)

    def __setitem__(self, key, value):  # 将键值对添加到 _dict 字典中
        self._dict[key] = value

    def __getitem__(self, key):  # 从 _dict 字典中获取注册的可调用对象
        return self._dict[key]

    def __contains__(self, key):  # 检查给定的注册名是否存在于 _dict 字典中
        return key in self._dict

    def __str__(self):  # 返回 _dict 字典的字符串表示
        return str(self._dict)

    def keys(self):  # 返回 _dict 字典中的所有键
        return self._dict.keys()

    def values(self):  # 返回 _dict 字典中的所有值
        return self._dict.values()

    def items(self):  # 返回 _dict 字典中的所有键值对
        return self._dict.items()


if __name__ == "__main__":
    register_obj = Register()
    
    @register_obj  # 不用再 register_obj.register 了
    def fn1_add(a, b):
        return a + b
    
    @register_obj  # 不用再 register_obj.register 了
    def fn2_subject(a, b):
        return a - b
    
    @register_obj  # 不用再 register_obj.register 了
    def fn3_multiply(a, b):
        return a * b
    
    @register_obj  # 不用再 register_obj.register 了
    def fn4_divide(a, b):
        return a / b
    
    # 我们再重复定义一个函数
    @register_obj  # 不用再 register_obj.register 了
    def fn2_subject(a, b):
        return b - a
    
    # 尝试使用 register 方法注册不可调用的对象
    try:
        register_obj("传入字符串，它是不可调用的")
    except Exception as e:
        print(f"报错啦: {e}")

    print("\n所有函数均已注册!\n")
    
    # 我们查看一个注册器中有哪些元素
    print(f"\033[34mkey\t\tvalue\033[0m")
    for k, v in register_obj.items():  # <=> for k, v in register_obj._dict.items()
        print(f"{k}: \t{v}")

# 4. MMCV 中的 Registry

In [8]:
from mmcv.utils import Registry

class Registry:
    # 构造函数
    def __init__(self, name, build_func=None, parent=None, scope=None):
        """
        name (str): 注册器的名字
        build_func(func): 从注册器构建实例的函数句柄
        parent (Registry): 父类注册器
        scope (str): 注册器的域名
        """
        self._name = name
        
        # 使用 module_dict 管理字符串到类的映射 {'str': class}
        self._module_dict = dict()
        self._children = dict()

In [11]:
import mmcv
# 实例化一个Registry来管理模型
MODELS = mmcv.utils.Registry("myModels")

# 方式1: 在类的创建过程中，使用函数装饰器进行注册(推荐)
@MODELS.register_module
class ResNet:
    def __init__(self, depth):
        self.depth = depth
        print(f"初始化 ResNet-{self.depth}...")
        
# 方式2: 完成类的创建后, 再显式调用register_module进行注册 (不推荐)
class FPN:
    def __init__(self, in_channel):
        self.in_channel = in_channel
        print(f"初始化FPN网络, 输入通道数为: {self.in_channel}")

AttributeError: 'NoneType' object has no attribute '__name__'

In [12]:
from mmcv.utils import Registry
CONVERTERS = Registry("converter")

AttributeError: 'NoneType' object has no attribute '__name__'